<a href="https://colab.research.google.com/github/3dsf/BlendGAN/blob/main/BlendGAN_3dsf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BlendGAN 
Runs examples from 
https://github.com/onion-liu/BlendGAN

*colab notebook adaptation by u/3dsf*

## Use GPU Runtime

In [ ]:
try :
  import torch
  torch.cuda.is_available()
  print('CUDA and TORCH found')  #not sure if next line might trigger a usage limit sooner...
  #print('CUDA device: ',torch.cuda.get_device_name(0))
except :
  print("\n\n\nPlease enable GPU runtime\n\n\n")
  #add in stop colab exectution...

## Install & Setup

In [ ]:
!git clone https://github.com/onion-liu/BlendGAN.git
!pip install gdown ninja

import os
from google.colab import files
pathBG = '/content/BlendGAN/'

In [ ]:
cpBlendGAN = os.path.join(pathBG, 'pretrained_models/blendgan.pt')
cpPSP      = os.path.join(pathBG, 'pretrained_models/psp_encoder.pt')
cpStyle    = os.path.join(pathBG, 'pretrained_models/style_encoder.pt')

if os.path.exists(cpBlendGAN) == False :
  !gdown --id 1eF04jKMLAb9DvzI72m8Akn5ykWf3EafE -O $cpBlendGAN
if os.path.exists(cpPSP) == False : 
  !gdown --id 14nevG94hNkkwaoK5eJLF1iv78cv5O8fN -O $cpPSP
if os.path.exists(cpStyle) == False :
  !gdown --id 1EaM0ZYsAMdPkbRz0smLNIlJ1rxVAhbEz -O $cpStyle

# Generate image pairs with random face codes
* First run will download vgg19 and will take longer



## latent-guided generation
* *uses ./test_imgs/face_imgs and ./test_imgs/style_imgs (coded into generate_image_pairs.py)*

In [ ]:
N_PICS = 8

###

os.chdir(pathBG)
pathOutputLatentGuided = os.path.join(pathBG, 'results/generated_pairs/latent_guided/')
!python generate_image_pairs.py --size 1024 --pics $N_PICS --ckpt $cpBlendGAN --outdir $pathOutputLatentGuided

os.chdir(pathOutputLatentGuided)
tarOutLG = os.path.join(pathBG, 'latentGuided.tar.gz')
!tar -czvf $tarOutLG *
files.download(tarOutLG)



## reference-guided generation

* *uses ./test_imgs/face_imgs  (coded into generate_image_pairs.py)*

In [ ]:
N_PICS = 8
refStyleImage = os.path.join(pathBG, 'test_imgs/style_imgs/100036.png')

###

os.chdir(pathBG)
pathOutputReferenceGuided = os.path.join(pathBG, '/results/generated_pairs/reference_guided/')
!python generate_image_pairs.py --size 1024 --pics $N_PICS --ckpt $cpBlendGAN --style_img $refStyleImage --outdir $pathOutputReferenceGuided

os.chdir(pathOutputReferenceGuided)
tarOutRG = os.path.join(pathBG, 'referenceGuided.tar.gz')
!tar -czvf $tarOutRG *
files.download(tarOutRG)



# Style tranfer with given face images

In [ ]:
pathStyleImgs = '/content/BlendGAN/test_imgs/style_imgs/'
pathInputImgs = '/content/BlendGAN/test_imgs/face_imgs/'
pathOutputStyleTransfer = '/content/BlendGAN/results/style_transfer/'

os.chdir(pathBG)
!python style_transfer_folder.py --size 1024 --ckpt $cpBlendGAN --psp_encoder_ckpt $cpPSP --style_img_path $pathStyleImgs --input_img_path $pathInputImgs --outdir $pathOutputStyleTransfer

os.chdir(pathOutputStyleTransfer)
tarOutST = os.path.join(pathBG, 'styleTransfer.tar.gz')
!tar -czvf $tarOutST *
files.download(tarOutST)

# ~Generate interpolation videos~
#### Crashes when writing video in colab and during local testing (RAM overflow)

*uncomment next cell to try*

In [ ]:
#os.chdir(pathBG)
#!python gen_video.py --size 1024 --ckpt $cpBlendGAN --psp_encoder_ckpt $cpPSP --style_img_path ./test_imgs/style_imgs/ --input_img_path ./test_imgs/face_imgs/ --outdir results/inter_videos/